
























# 2D Localization for a holonomous mobile robot
<img src="images/ProblemStatement.jpg" alt="Problem Statement Image" width="500" style="float: center"/>

Consider a robot moving in 2D Space, there are two state variables $x$ and $y$

Let blue circle be the position of robot and red stars be the available markers in the environment. All markers are identical

The Robot is equipped with a binary sensor that detects the marker with some probability, similar to our 1D example

The origin is at lower left corner and the state space can be written as:

$domain(X) = \{(x,y)\in \Re^2 | 0 <= x <= 10, 0 <=y <=10\}$

Robot doesn't know its initial location, hence we assign a uniform belief initially to all positions

Robot's belief will be represented using a 2D matrix

We will assume that the Robot's world wraps around like a disk

Let, initial _ground truth_ position for $(x,y) = (3,4)$

Note that origin of each cell will be located at bottom-left corner of each cell

This should be clear from the initial ground postion, the way it is defined

Similary, marker positions are defined as

$M(x,y) = \begin{bmatrix}
4 & 4 \\
4 & 6 \\
4 & 8 \\
6 & 8 \\
8 & 8
\end{bmatrix}$

**Your task is to write the appropriate state transition and measurement probability functions governing the 2D space of Robot, so that a underlying Bayes filter can use these to localize the robot.**

## State Transition Probability
<img src="images/StateTransitionRight.jpg" alt="hello!" title="State Transition Image" width="400" style="align: left"/>

<img src="images/StateTransitionUp.jpg" title="State Transition Image" width="400" style="align: right"/>

Let's assume our robot at any instant can make either of two different types of motion
- Sideways to the right: $u_r$
- Upside: $u_u$

We will assume that the robot makes a shift of 1 cell at every state transition or motion update

Let's define the state transition probabilities as depicted in the figure above

The state can overshoot towards the direction of motion and also sideways

The transition by one cell has a probability of 0.6 and all all others 0.1

Your task is to write two methods to update the existing belief to get the prior belief based on two motion types

**Note** that the belief is a numpy array with indexing of (row,col) 

_Hint_ To make the implmentation simpler, think about updating probability values for each cell based on where it can get the state transition from

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Polygon
from matplotlib.collections import PatchCollection

In [2]:
def motion_update_u_r(px_rt, u):
    """
    Update belief using motion model in right direction
    
    :param px : numpy array of size n x n representing belief
    :param u : control input u in the right direction
    :return px: Prior Belief after motion update
    """
    # Your code here
    r,c = px_rt.shape
    for i in range(px_rt.shape[0]):      
        for j in range(px_rt.shape[1]):
            px_rt[i,(j+u)%c]*=0.6
            px_rt[i,(j+u-1)%c]*=0.1
            px_rt[i,(j+u+1)%c]*=0.1
            px_rt[(i-u)%r,(j+u)%c]*=0.1
            px_rt[(i+u)%r,(j+u)%c]*=0.1
            
    return px_rt

In [3]:
def motion_update_u_u(px_up, u):
    """
    Update belief using motion model in up direction
    
    :param px : numpy array of size n x n representing belief
    :param u : control input u in the up direction
    :return px: Prior Belief after motion update
    """
    # Your code here
    r,c = px_up.shape
    for i in range(px_up.shape[0]):      
        for j in range(px_up.shape[1]):
            px_up[(i+u)%r,j]*=0.6
            px_up[(i+u-1)%r,j]*=0.1
            px_up[(i+u+1)%r,j]*=0.1
            px_up[(i+u)%r,(j-u)%c]*=0.1
            px_up[(i+u)%r,(j+u)%c]*=0.1   
    return px_up

## Measurement Update
<img src="images/MeasurementProbability.jpg" title="Measurement Probability Image" width="400" style="align: right"/>

Let's define a simple measurement/observation likelihood function

The probability that the robot detects a marker at $(x,y) \in M(x,y) = 0.7$

Let, $M(x-1,y) = \begin{bmatrix}
3 & 4 \\
3 & 6 \\
3 & 8 \\
5 & 8 \\
7 & 8
\end{bmatrix}$

The probability that the robot detects a marker at $(x,y) \in M(x-1, y) = 0.25$

Rest of the locations in state space let the probability be 0.05

This is as shown in the diagram above

Recall,
$P(x|z) = \frac{P(z|x) P(x)}{P(z)} = \frac{P(z|x) p(x)}{\sum_x P(z|x) P(x)}$

$P(x|z) = \frac{likelihood x prior}{evidence}$

We can create a likelihood map given the Marker positions & then write the equation for measurement update

In [4]:
def obsv_likelihood(nrows, ncols):
    """
    Create the obsv likelihood p(z|x)
    
    :param nrows: Number of rows in the state space
    :param ncols: Number of cols in the state space
    :return pzx: Likelihood map (unnormalized) given the marker positions and sensor model
    """
    pzx = np.zeros((nrows, ncols))
    
    # Your code here      
                
    x_loc=0
    x_1_loc=0
    for i in range(pzx.shape[0]):      
        for j in range(pzx.shape[1]):
            if(x_loc<len(markerPos)):
                r,c = markerPos[x_loc]
            if(x_1_loc<len(markerPos_x_1)):
                r1,c1 = markerPos_x_1[x_1_loc]            
            if(i == r1) and (j==c1):
                pzx[i][j] = 0.25
                x_1_loc+=1
            elif(i == r) and (j==c):
                pzx[i][j] = 0.7
                x_loc+=1           
            else:
                pzx[i][j] = 0.05
    #print("obs_LL {}".format(pzx))
    return pzx


In [5]:
def measurement_update(px):
    """
    Measurement Update function for Bayesian Filter Algorithm
    
    Return the normalized posterior distribution function
    
    :param px: Prior belief as numpy array of size nxn
    :return px: Normalized posterior belief
    """
    # Call your likelihood function here and find normalized posterior belief
    row,col = px.shape
    LLhoodMap = obsv_likelihood(row,col)
    # Your code here
    px_new=np.dot(LLhoodMap,px)
    #print("px_new {}".format(px_new))
    # Normalizing
    px_new = px_new / np.sum(px_new)
    
    return px_new
    

In [6]:
# Apply Bayes Filter Algorithm
# You can choose to experiment with this part, but no change should be necessary for the challenge
meas_upd_iter = [0,1,3,5,7,9]
right_state_trans_iter = [1,2,7,8,9]
up_state_trans_iter = [3,4,5,6]

z_true_det_marker = 0.7
z_false_det_marker = 0.25

markerPos = np.array(([4,4],[4,6],[4,8],[6,8],[8,8]))
markerPos_x_1 = np.array(([3,4],[3,6],[3,8],[5,8],[7,8]))

u = 1
n = 10
px = np.ones((n,n)) / (n * n)

iter=10

# Make the first measurement
beliefs = [px]
px = measurement_update(px)
#print("First measurement {}".format(px))
# Run Bayes Filter for 10 iterations
for i in range(1, iter):
    # State Transition
    if i in right_state_trans_iter:
        px = motion_update_u_r(px, u)
        #print("right {}".format(px))
    elif i in up_state_trans_iter:
        px = motion_update_u_u(px, u)
        #print("up {}".format(px))
    # Measurement Update
    if i in meas_upd_iter:
        px = measurement_update(px)
    
    beliefs.append(px)

In [7]:
%matplotlib widget

In [8]:
# Simulation -  You don't have to modify this section, 
# you can choose to use code from here to debug your outputs if you like
from ipywidgets import interact, IntSlider

fig, ax = plt.subplots()

curr_belief = beliefs[0]
im = plt.imshow(curr_belief, cmap='gray_r', vmin=0, vmax=1, extent=[0,10,0,10], origin='lower')
ax.set_xticks(np.arange(0,n))
ax.set_yticks(np.arange(0,n))
cbar = fig.colorbar(im, ticks=[0,1])
cbar.ax.set_yticklabels(['p(x)=0', 'p(x)=1'])
plt.grid()
ax.set_box_aspect(1)

# Add Markers
patches_marker = []
points= [[4,4.7], [5,4.7], [4,4], [4.5, 5], [5, 4], [4,4.7]]
patches_marker.append(Polygon(points))
points= [[4,6.7], [5,6.7], [4,6], [4.5, 7], [5, 6], [4,6.7]]
patches_marker.append(Polygon(points))
points= [[4,8.7], [5,8.7], [4,8], [4.5, 9], [5, 8], [4,8.7]]
patches_marker.append(Polygon(points))
points= [[6,8.7], [7,8.7], [6,8], [6.5, 9], [7, 8], [6,8.7]]
patches_marker.append(Polygon(points))
points= [[8,8.7], [9,8.7], [8,8], [8.5, 9], [9, 8], [8,8.7]]
patches_marker.append(Polygon(points))

p_m = PatchCollection(patches_marker, facecolor='r', edgecolor='k')
ax.add_collection(p_m)
    
robot_gt_poses = [[3.5,4.5], [4.5, 4.5], [4.5, 5.5], [4.5, 6.5], [4.5, 7.5], [4.5, 8.5], [5.5, 8.5], [6.5, 8.5], [7.5, 8.5], [8.5, 8.5]]
patches_robot = []

# Show the belief updates
def show_beliefs(step):
    plt.cla()
    ax.add_collection(p_m)
    
    ax.set_xticks(np.arange(0,n))
    ax.set_yticks(np.arange(0,n))
    plt.grid()
    ax.set_box_aspect(1)
    
    curr_belief = beliefs[step]
    im = plt.imshow(curr_belief, cmap='gray_r', vmin=np.min(px), vmax=np.max(px), extent=[0,10,0,10], origin='lower')
    
    # Add Gt Robot Pose For Reference
    patches_robot.clear()
    curr_gt_pose = robot_gt_poses[step]
    patches_robot.append(Circle((curr_gt_pose[0], curr_gt_pose[1]), 0.4))
    p_r = PatchCollection(patches_robot, facecolor='b', edgecolor='k')
    ax.add_collection(p_r)
    plt.show()
    plt.title(f'Time Step {step}')
    
interact(show_beliefs, step=IntSlider(value=0, max=len(beliefs)-1));

interactive(children=(IntSlider(value=0, description='step', max=9), Output()), _dom_classes=('widget-interact…

## Expected Output
<img src="images/Expected2DBayesFilterOutput.jpg" title="Final Output Image" width="400" style="float: center"/>

At the end of 10 iterations your output should look something like above

## Extra exercise
Feel free to extend the motion model and measurement model and add more iterations to further improve robot's belief